<a href="https://colab.research.google.com/github/dsba6010-llm-applications/los-tres-amigos/blob/main/Syllabi_Crawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example Beautiful Soup Spider

* https://chatgpt.com/share/66ecea93-c990-8007-8258-538cd80c00f5
* https://docs.scrapy.org/en/latest/topics/architecture.html

Tried Scrapy and could not get it to work


In [ ]:
!pip install scrapy
!pip install PyPDF2
!pip install crochet
# Install the Python SDK
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.3/254.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 40.3 MB/s eta 0:00:00


## Imports

In [ ]:
# Gemini
import google.generativeai as genai
from google.colab import userdata
from google.generativeai.types import HarmCategory, HarmBlockThreshold
import time

# Core Python Libraries
import json
import os
import sys
import requests

from datetime import date, datetime
from scipy.optimize import minimize

# Data Science Specific Libraries
import pandas as pd
import numpy as np

# Google Collab Goodies
from IPython.display import display_markdown, display_html
from google.colab import data_table
data_table.enable_dataframe_formatter()
from google.colab import drive

import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from PIL import Image, ImageOps
from PIL.ExifTags import TAGS

from scipy.spatial.distance import pdist, squareform
import sklearn.metrics.pairwise as smp

from os import symlink
import re
import requests
from bs4 import BeautifulSoup
import json

import requests
import PyPDF2
from io import BytesIO
from copy import deepcopy

# Google Collab Goodies
from IPython.display import display_markdown, display_html
from google.colab import data_table
data_table.enable_dataframe_formatter()
from google.colab import drive

## Mount Drive

In [ ]:
SYLLABI_DOCS= '/content/drive/MyDrive/Syllabi'
# Mount The Data Folder
drive.mount('/content/drive')

Mounted at /content/drive


## BeautifulSoup

* https://chatgpt.com/share/66ecea93-c990-8007-8258-538cd80c00f5
* https://chatgpt.com/share/66f640a6-97ec-8007-96a4-7c31741707c7

In [ ]:


# This is the element that has the Syllabi links
def find_basic_list_ul(soup):
    return soup.find('ul', class_='basic-list')

# Function to scrape a for links that meet a specific criteria
def scrape_webpage_for_links(url,callback):
    sylinks = list()
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        ul = callback(soup)
        # Extract links
        links = [(a.get_text(), a['href']) for a in ul.find_all('a', href=True)]

        print('\nLinks:')
        for link_text, link_url in links:
            sylinks.append(dict({"link": link_url, "title": link_text.strip()}))
    else:
        print(f'Failed to retrieve the webpage. Status code: {response.status_code}')
    return sylinks

# Function to scrape a webpage not using this anymore
def scrape_webpage(url):
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract the title
        title = soup.title.string if soup.title else 'No Title'

        # Extract paragraphs
        paragraphs = [p.get_text() for p in soup.find_all('p')]

        # Extract links
        links = [(a.get_text(), a['href']) for a in soup.find_all('a', href=True)]

        # Print the results
        print(f'Title: {title}')
        print('Paragraphs:')
        for para in paragraphs:
            print(para)

        print('\nLinks:')
        for link_text, link_url in links:
            print(f'{link_text}: {link_url}')
    else:
        print(f'Failed to retrieve the webpage. Status code: {response.status_code}')

# Get content and put it into memory
def download_to_memory(url):
    try:
        # Send the GET request with streaming enabled
        response = requests.get(url, stream=True)

        if response.status_code == 200:
            # Create an in-memory bytes buffer
            file_in_memory = BytesIO()

            # Read and accumulate chunks into the memory buffer
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    file_in_memory.write(chunk)

            # Optionally, reset the pointer to the beginning of the stream
            file_in_memory.seek(0)

            print(f"Downloaded {url} into memory")
            return file_in_memory  # Return the in-memory file-like object

        else:
            print(f"Failed to download {url}: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        return None

# Function to download and extract text from a PDF
def extract_pdf_content(pdf_url, filename=None):
    # Send a GET request to the PDF URL
    response = requests.get(pdf_url)

    # Check if the request was successful
    if response.status_code == 200:
        # Open the PDF file from the response content
        pdf_file = BytesIO(response.content)

        if filename:
          with open(f"{SYLLABI_DOCS}/{filename}", "wb") as f:
              f.write(pdf_file.read())
          print(f"PDF saved as '{filename}'")


        # Create a PDF reader object
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        # Extract text from each page
        text = ''
        for page in pdf_reader.pages:
            text += page.extract_text() or ''  # Append text, handle None

        # Print the extracted text
        return text
    else:
        print(f'Failed to retrieve the PDF. Status code: {response.status_code}')



In [ ]:

# URL to scrape
url = 'https://dsba.charlotte.edu/syllabi/'
document_title = 'DSBA'
sylinks = scrape_webpage_for_links(url,find_basic_list_ul)
doclinks = list()
for linkitem in sylinks:
    try:
        display_markdown(f"### Fetching {linkitem['title']}", raw=True)
        sub_page_response = requests.get(linkitem["link"])
        #print(f"Fetched {linkitem['link']}")
        if sub_page_response.status_code == 200:
            sub_soup = BeautifulSoup(sub_page_response.content, 'html.parser')
            #print(f"Parsed {linkitem['link']}")
            # Find all PDF or DOCX links in the sub-pages
            file_links = [(a_tag.get_text(), a_tag['href']) for a_tag in sub_soup.find_all('a', href=True)
            if re.search(r'\.(pdf|docx)$', a_tag['href']) and document_title.lower() in a_tag.text.lower()]
            display_markdown(f" -- _Found {len(file_links)} files in {linkitem['link']}_",raw=True)
            for link_text, link_url in file_links:
                # TODO:  fix this to handle docx
                content = extract_pdf_content(link_url, filename=link_text)
                # print(content)
                doclinks.append(dict({"link": link_url, "filename": link_text, "parent":linkitem["link"], "title": linkitem["title"], "content": content}))
    except Exception as e:
        print(f"Error fetching or parsing link {linkitem['link']}: {e}")

#print(json.dumps(doclinks,indent=2))
df=pd.DataFrame(doclinks)
display(df.drop(columns=['content']))




Links:


### Fetching DSBA 5122 Visual Analytics and Storytelling, Fall 2024 – Robert Fox

 -- _Found 1 files in https://dsba.charlotte.edu/2024/10/10/dsba-hcip-6160-database-systems-for-data-scientists-fall-2024-robert-fox/_

PDF saved as 'DSBA_HCIP-5122-Visual-Analytics-and-Storytelling-Fall-2024-Robert-Fox.pdf'


### Fetching DSBA 6010 Applications of Large Language Models, Fall 2024 – Ryan Wesslen

...

PDF saved as 'Spring 2023 DSBA and HCIP 6400 – Doug Hague.pdf'


### Fetching DSBA 6500 Complex Adaptive Systems, Spring 2024 – Mirsad Hadzikadic

 -- _Found 1 files in https://dsba.charlotte.edu/2024/03/12/dsba-6500-complex-adaptive-systems-mirsad-hadzikadic/_

PDF saved as 'DSBA 6500 Complex Adaptive Systems – Mirsad Hadzikadic'


,link,filename,parent,title
0,https://dsba.charlotte.edu/wp-content/uploads/...,DSBA_HCIP-5122-Visual-Analytics-and-Storytelli...,https://dsba.charlotte.edu/2024/10/10/dsba-hci...,"DSBA 5122 Visual Analytics and Storytelling, F..."
1,https://dsba.charlotte.edu/wp-content/uploads/...,DSBA-6010-Applications-of-Large-Language-Model...,https://dsba.charlotte.edu/2024/10/10/dsba-601...,DSBA 6010 Applications of Large Language Model...
2,https://dsba.charlotte.edu/wp-content/uploads/...,DSBA 6010 Applied Computer Vision – Rick Chakra,https://dsba.charlotte.edu/2024/03/12/dsba-601...,"DSBA 6010 Applied Computer Vision, Spring 2024..."
3,https://dsba.charlotte.edu/wp-content/uploads/...,Damien_Patrick_Williams_-_DSBA_6010_-_26287 – ...,https://dsba.charlotte.edu/dsba-6010-disabilit...,"DSBA 6010 Disability, Technology, & Artificial..."
4,https://dsba.charlotte.edu/wp-content/uploads/...,DSBA6010-GEOG6030 Syllabus SP 2023 v30 – Jean-...,https://dsba.charlotte.edu/dsba-6010-geospatia...,"DSBA 6010 Geospatial Data Analytics, Spring 20..."
5,https://dsba.charlotte.edu/wp-content/uploads/...,DSBA-6010-Model-Risk-Management-Fal,https://dsba.charlotte.edu/2024/10/10/dsba-601...,DSBA 6010 Special Topic: Model Risk Management...
6,https://dsba.charlotte.edu/wp-content/uploads/...,DSBA-6010-Fall-2024-Steven-Jordan.pdf,https://dsba.charlotte.edu/2024/10/10/dsba-601...,DSBA 6010 Special Topics: Building & Leading D...
7,https://dsba.charlotte.edu/wp-content/uploads/...,DSBA6100-U90_MBAD7090-U92_Spring 2023_Blaine N...,https://dsba.charlotte.edu/dsba-6100-big-data-...,DSBA 6100 Big Data Analytics for Competitive A...
8,https://dsba.charlotte.edu/wp-content/uploads/...,DSBA-6112-Graduate-Econometrics-Fall-2024-Kell...,https://dsba.charlotte.edu/2024/10/10/dsba-eco...,"DSBA 6112 Graduate Econometrics, Fall 2024 – K..."
9,https://dsba.charlotte.edu/wp-content/uploads/...,DSBA6112-U90_202310 – Kelly Vosters.pdf,https://dsba.charlotte.edu/dsba-6112-graduate-...,"DSBA 6112 Graduate Econometrics, Spring 2023 –..."


# Getting metadata from titles

In [ ]:
prompt="""
Analyze the title at the end of this prompt and provide the following in json format if available, using the following keys:
  - course_title
  - course_number
  - instructor
  - semester
  - section

please make sure that the json is valid json escape quotes in necessary
please do not wrap the json with markdown to indicate that it is json
please make sure that the json labels match the labels below exactly
Title: {title}

Here are some examples:
DSBA 5122 Visual Analytics & Storytelling, Fall 2022 – WenWen Dou
{{
  "course_title": "Visual Analytics & Storytelling",
  "course_number": "5122",
  "instructor": "WenWen Dou",
  "semester": "Fall 2022",
  "section": ""
}}

DSBA 6010 Applied Computer Vision – Rick Chakra
{{
  "course_title": "Applied Computer Vision",
  "course_number": "6010",
  "instructor": "Rick Chakra",
  "semester": "",
  "section": ""
}}
DSBA 6188 Text Mining and Information Retrieval – Xi (Sunshine) Niu
{{
  "course_title": "Text Mining and Information Retrieval",
  "course_number": "6188",
  "instructor": "Xi (Sunshine) Niu",
  "semester": "",
  "section": ""
}}
DSBA-6165-Artificial Intelligence and Deep Learning Section 1
{{
  "course_title": "",
  "course_number": "6165",
  "instructor": "",
  "semester": "",
  "section": "1"
}}
  """

GEMINI_MODEL_VERSION='models/gemini-1.5-flash'
gemini_key = userdata.get('GeminiAPI')
genai.configure(api_key=gemini_key)
model=genai.GenerativeModel(GEMINI_MODEL_VERSION)
corpus=list()
for doc in doclinks:
    #print(f"Processing {doc['title']}\n{prompt.format(**doc)}")
    print(f"Processing {doc['title']}")
    time.sleep(5)  ## Throttle Needed
    response = model.generate_content(prompt.format(**doc))
    corpus_doc = deepcopy(doc)
    corpus_doc.update(json.loads(response.text))
    corpus.append(corpus_doc)

corpus_df=pd.DataFrame(corpus)
display(corpus_df.drop(columns=['link','parent','content']))

syllabi_fp = open(SYLLABI_DOCS + '/syllabi.json', 'w')
syllabi_fp.write(corpus_df.to_json(orient='records'))
syllabi_fp.close()

Processing DSBA 5122 Visual Analytics and Storytelling, Fall 2024 – Robert Fox
Processing DSBA 6010 Applications of Large Language Models, Fall 2024 – Ryan Wesslen
Processing DSBA 6010 Applied Computer Vision, Spring 2024 – Rick Chakra
Processing DSBA 6010 Disability, Technology, & Artificial Intelligence, Spring 2023 – Damien Williams
Processing DSBA 6010 Geospatial Data Analytics, Spring 2023 – Jean Claude Thill
Processing DSBA 6010 Special Topic: Model Risk Management, Fall 2024 – Robert Fox
Processing DSBA 6010 Special Topics: Building & Leading Data Organizations, Fall 2024 – Steven Jordan
Processing DSBA 6100 Big Data Analytics for Competitive Advantage, Spring 2023 – Daryl Nashold
Processing DSBA 6112 Graduate Econometrics, Fall 2024 – Kelly Vosters
Processing DSBA 6112 Graduate Econometrics, Spring 2023 – Kelly Vosters
Processing DSBA 6112 Graduate Econometrics, Spring 2024 – Kelly Vosters
Processing DSBA 6115 Statistical Learning with Big Data, Fall 2024 – Jiancheng Jiang
Proc

,filename,title,course_title,course_number,instructor,semester,section
0,DSBA_HCIP-5122-Visual-Analytics-and-Storytelli...,"DSBA 5122 Visual Analytics and Storytelling, F...",Visual Analytics and Storytelling,5122,Robert Fox,Fall 2024,
1,DSBA-6010-Applications-of-Large-Language-Model...,DSBA 6010 Applications of Large Language Model...,Applications of Large Language Models,6010,Ryan Wesslen,Fall 2024,
2,DSBA 6010 Applied Computer Vision – Rick Chakra,"DSBA 6010 Applied Computer Vision, Spring 2024...",Applied Computer Vision,6010,Rick Chakra,Spring 2024,
3,Damien_Patrick_Williams_-_DSBA_6010_-_26287 – ...,"DSBA 6010 Disability, Technology, & Artificial...","Disability, Technology, & Artificial Intelligence",6010,Damien Williams,Spring 2023,
4,DSBA6010-GEOG6030 Syllabus SP 2023 v30 – Jean-...,"DSBA 6010 Geospatial Data Analytics, Spring 20...",Geospatial Data Analytics,6010,Jean Claude Thill,Spring 2023,
5,DSBA-6010-Model-Risk-Management-Fal,DSBA 6010 Special Topic: Model Risk Management...,Special Topic: Model Risk Management,6010,Robert Fox,Fall 2024,
6,DSBA-6010-Fall-2024-Steven-Jordan.pdf,DSBA 6010 Special Topics: Building & Leading D...,Special Topics: Building & Leading Data Organi...,6010,Steven Jordan,Fall 2024,
7,DSBA6100-U90_MBAD7090-U92_Spring 2023_Blaine N...,DSBA 6100 Big Data Analytics for Competitive A...,Big Data Analytics for Competitive Advantage,6100,Daryl Nashold,Spring 2023,
8,DSBA-6112-Graduate-Econometrics-Fall-2024-Kell...,"DSBA 6112 Graduate Econometrics, Fall 2024 – K...",Graduate Econometrics,6112,Kelly Vosters,Fall 2024,
9,DSBA6112-U90_202310 – Kelly Vosters.pdf,"DSBA 6112 Graduate Econometrics, Spring 2023 –...",Graduate Econometrics,6112,Kelly Vosters,Spring 2023,
